In [1]:
!pip install py7zr

!pip install wandb
import os
!WANDB_API_KEY=$46c39bbac8b187eeb815386597ca25ba414d4c5e
os.environ['WANDB_API_KEY'] = '46c39bbac8b187eeb815386597ca25ba414d4c5e'
import wandb

In [3]:
import cv2
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings("ignore")
import py7zr
from py7zr import unpack_7zarchive
import shutil
import glob
import random as rd

import torch 
import torch.nn as nn 
from torch.utils.data import Dataset, DataLoader 
import torchvision 
import torchvision.transforms as transforms 
# from torchvision.io import read_image 
import torch.nn.functional as F 

import pandas as pd
from PIL import Image
import numpy as np
import sys
import os 
from pprint import pprint


In [4]:
!mkdir /kaggle/working/data
shutil.register_unpack_format('7zip', ['.7z'], unpack_7zarchive)
shutil.unpack_archive("../input/cifar-10/test.7z", "/kaggle/working/data/")
shutil.unpack_archive("../input/cifar-10/train.7z", "/kaggle/working/data/")

In [5]:
!ls /kaggle/working/data
train_csv = pd.read_csv('../input/cifar-10/trainLabels.csv')

In [6]:
train_csv

#### Dataloader

In [9]:
transform = transforms.Compose({
        transforms.ToTensor(),
        transforms.Normalize((0.5,0.5,0.5), (0.5, 0.5, 0.5)),
    })

class dataset(Dataset):

    classes = {'truck':0, 'airplane':1, 'automobile':2, 'bird':3, 'cat':4,
           'deer':5, 'dog':6, 'frog':7, 'horse':8, 'ship':9, }
        
    def __init__(self, train_csv):
        super(dataset, self).__init__()
        self.df = pd.read_csv(train_csv)
        self.label = self.df['label'].tolist()
        self.id = self.df['id'].tolist()
        self.classes = {'truck':0, 'airplane':1, 'automobile':2, 'bird':3, 'cat':4,
           'deer':5, 'dog':6, 'frog':7, 'horse':8, 'ship':9, }
       
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        root_dir = '/kaggle/working/data/train/'
        id_ = self.id[idx]
        image = os.path.join(root_dir, f'{id_}.png')
        image = np.array(Image.open(image))   # np.array()
        label = self.label[idx]
        label = self.classes[label]
        
        image = transform(image)
        
        return image, label
        
    

In [10]:
data = dataset("../input/cifar-10/trainLabels.csv")
train_dataset, val_dataset, = torch.utils.data.random_split(data, [45000, 5000], generator=torch.Generator().manual_seed(69))
# dataloader = DataLoader
train_loader = DataLoader(train_dataset, batch_size = 32)
val_loader = DataLoader(val_dataset, batch_size=32)

#### Train CNN model


In [18]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN,self).__init__()
    self.conv1 = nn.Conv2d(3, 64, 3)
#     self.pool  = nn.MaxPool2d(2,2)
    self.pool  = nn.MaxPool2d(4,4)
    self.norm  = nn.BatchNorm2d(64)
    self.conv2 = nn.Conv2d(64, 128, 3)
    self.fc1   = nn.Linear(128*28*28, 256)
    self.drop  = nn.Dropout(0.3)
    self.fc2   = nn.Linear(256, 128)
    self.fc3   = nn.Linear(128,10)

  def forward(self, x):
    x = (self.conv1(x))
    print("conv1 ", x.size())
    x = self.pool(x)
    print("pool ", x.size())
    x = F.relu(self.norm(x))
    print("norm ", x.size())
    x = F.relu(self.conv2(x))
    print("conv2 ",x.size())
    x = self.pool(x)
    print("pool ",x.size())
    x = torch.flatten(x,1)
    print("flatten ", x.size())
    x = F.relu(self.fc1(x))
    x = self.drop(x)
    print(x.size())
    x = F.relu(self.fc2(x))
    print(x.size())
    x = self.fc3(x)
    print(x.size())
    return x

In [19]:

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
model = CNN()
model.to(device)
history = []
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
loss_fn = nn.CrossEntropyLoss()
epochs = 20
train_losses = []
train_accuracy = []

for epoch in range(epochs):
    print(f"epoch {epoch}")
    for i, batch in enumerate(train_loader, 0):
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)
        # forward pass
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        acc = accuracy(outputs, labels)
        # set gradients to zero
        optimizer.zero_grad()
        # backward pass
        loss.backward()
        # update gradients
        optimizer.step()
        
        
        train_losses.append(loss)
        train_accuracy.append(acc)
        if(i%100==0):
            print(f"\tloss: {loss} \tacc: {acc}")

    result = {}
    result['train_loss'] = torch.stack(train_losses).mean().item()
    result["train_accuracy"] = torch.stack(train_accuracy).mean().item()
    pprint(result)
    
    history.append(result)


In [ ]:
# history

#### Currently the model is overfitting on train data. so we'll try and intoduce a validation set during training.

In [11]:
def accuracy(outputs, labels):
    _, pred = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(pred==labels)/len(pred))
    
class imageClassification(nn.Module):
    def __init__(self):
        super().__init__()
        
        pass
    
    def training_step(self, batch):
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = self(inputs)
        loss_func = nn.CrossEntropyLoss()
        loss = loss_func(outputs, labels)
        acc = accuracy(outputs, labels)
        return loss, acc
    
    def validation_step(self, batch):
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = self(inputs)
        loss_func = nn.CrossEntropyLoss()
        loss = loss_func(outputs, labels)
        acc = accuracy(outputs, labels)
        return {'loss': loss, "acc": acc}
    
    def validation_epoch(self, result):
        acc = [x['acc'] for x in result]
        losses = [x['loss'] for x in result]
        val_loss = torch.stack(losses).mean()
        val_acc = torch.stack(acc).mean()        
        return {'val_loss':val_loss, 'val_acc':val_acc}
    
    def end_of_epoch(self, result):
        print(f"train_accuracy: {result['train_acc']*100:.2f}  train_loss: {result['train_loss']*100:.2f}")
        print(f"val_accuracy:   {result['val_acc']*100:.2f}    val_loss: {result['val_loss']*100:.2f}")
        print('-----------------------------------------------------------------')

In [12]:
class CNN(imageClassification):
    def __init__(self):
        super(CNN, self).__init__()
        self.network = nn.Sequential(
            nn.Conv2d(3, 64, 3),
#             nn.nn.AvgPool2d(2),
            nn.MaxPool2d(2,2),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 128, 3),
            nn.MaxPool2d(2,2),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Flatten(end_dim=-1),
            nn.Linear(128*6*6, 256),
            nn.Dropout(0.4),
            nn.ReLU(),            
            nn.Linear(256, 128),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(128,10)
        )
        
    def forward(self, x):
        return self.network(x)

In [13]:
@torch.no_grad()
def evaluate(model, dataloader):
    model.eval()
    result = [model.validation_step(batch) for batch in dataloader]
    return model.validation_epoch(result)
    

In [14]:
def log_results_wandb(result):
    wandb.log({"train loss":result['train_loss'],
               "train acc": result['train_acc'],
               "val acc": result['val_acc'],
               "val loss": result["val_loss"]})

In [41]:
def train(model, train_loader, val_loader, epochs=50, learning_rate=0.001):
    history = []
#     optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    model.train()
    for _, epoch in enumerate(range(epochs)):
        print(f"Epoch {_}: ")
        train_losses = []
        train_acc = []
        
        for i, batch in enumerate(train_loader, 0):
            loss, acc = model.training_step(batch)
            loss.backward()    
            optimizer.step()
            optimizer.zero_grad()
            
            if(i%100==0):
                print(f"\tacc: {acc} \tloss: {loss}")
            
            train_losses.append(loss)
            train_acc.append(acc)
            
        result = evaluate(model, val_loader)
        result["train_loss"] = torch.stack(train_losses).mean()
        result["train_acc"] = torch.stack(train_acc).mean()
        
        log_results_wandb(result)
        
        history.append(result)
        model.end_of_epoch(result)
        
    return model, history
        

In [46]:
config = {
    'model':'CNN (2 layers)',
    'architecture': '2*CPoolBnormR-Flat-LDrop-LDropR-Softmax',
    'epochs': 50,
    'batch_size': 32,
    'optimizer': 'Adam',
    'learning_rate': 0.00001,
    'CNN1': 64,
    'CNN2': 128,
    'Pool': ['max', [2, 2]],
    'BatchNorm': True,
    'dropout': [0.5, 0.5],
    'dense1_units': 128*2*2,
    'dense2_units': 256,
}

In [47]:
model = CNN()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

run = wandb.init(config=config, project='cifar', group='CNN model', notes='2 Conv layer cnn model')
wandb.watch(model) 

model, history = train(model, train_loader, val_loader, 50, 0.00001)

In [48]:
wandb.finish()

## Architecture Performance logging (manual) 
after 10 epochs: <br>
lr: 0.01
1. CPBR-CPR-F-LDrR-LR-Sm 
    - train_accuracy: 94.60    train_loss: 16.06
    - val_accuracy:   68.97    val_loss: 162.68 
    - high overfiting
2. CBR-CR-F-LDrR-LR-Sm
    - train_accuracy: 98.34  train_loss: 5.42
    - val_accuracy:   62.62    val_loss: 244.06
    - even higher overfitting
3. CBR-CBR-F-LDrR-LR-Sm
    - train_accuracy: 94.65  train_loss: 17.22
    - val_accuracy:   61.25    val_loss: 191.07
   
4. CPBR-CPBR-F-LDrR-LDrR-Sm
    - train_accuracy: 76.00  train_loss: 68.23
    - val_accuracy:   64.55    val_loss: 118.88
5. CPBR-CPBR-F-LDrR-LDrR-Sm (4 and 2 pools)
    - train_accuracy: 82.60  train_loss: 48.68
    - val_accuracy:   65.57    val_loss: 129.00

### Further analysis done in wandb: <a href="https://wandb.ai/mindadeepam/cifar?workspace=user-mindadeepam">link</a>